In [ ]:
# Setup
import os
import sys
import pandas as pd
from scipy.stats import chi2_contingency, f_oneway
import numpy as np

# Imports
sys.path.append(os.path.abspath(".."))
from Answercategories import question_orders
from lists import survey_questions
from Helper_funtions import lade_excel, test_significance

# Daten laden
file_path = "c:/Users/tills/Bachelor-Code/Analyse_Files/Data/Fertige Tabelle.xlsx"
df = lade_excel(file_path)

# Fragepaare generieren, ohne doppelte Kombinationen
from itertools import combinations
question_pairs = list(combinations(survey_questions, 2))

for q1, q2 in question_pairs:
    try:
        print(f"\n📊 Vergleich: {q1} ➜ {q2}")

        data = df[[q1, q2]].dropna()
        cross = pd.crosstab(data[q1], data[q2], normalize='index') * 100

        # Sortiere beide Achsen falls Reihenfolge definiert ist
        if q1 in question_orders:
            for val in question_orders[q1]:
                if val not in cross.index:
                    cross.loc[val] = 0
            cross = cross.loc[question_orders[q1]]

        if q2 in question_orders:
            for val in question_orders[q2]:
                if val not in cross.columns:
                    cross[val] = 0
            cross = cross[question_orders[q2]]

        print(cross.round(1).to_string())

        # Signifikanztest
        p = test_significance(df, q2, q1)
        sign = "✅ significant" if p < 0.05 else "❌ not significant"
        print(f"   → p-value: {p:.4f} → {sign}")

    except Exception as e:
        print(f"❌ Fehler bei {q1} vs {q2}: {e}")
